In [1]:
# nse scratchpad initiation
from helper import *
from nse_func import *

# Do assignments
a = assign_var('nse')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

In [2]:
ib =  get_connected('nse', 'live')

Peer closed connection
clientId 1 already in use?
API connection failed: CancelledError()


In [3]:
# get the remaining quantities
dfrq = remqty_nse(ib)

dft = pd.read_pickle(fspath+'targets.pickle')

dft = dft.assign(remqty=dft.symbol.map(dfrq.remqty.to_dict()))   # remaining quantities
dft = dft[dft.remqty > 0].reset_index(drop=True) # remove blacklisted (remqty <= 0)

commissionReport: No execution found for CommissionReport(execId='00016710.5cdccbad.01.01', commission=26.264, currency='INR')


In [ ]:
dft = targets(ib)
dft

In [ ]:
dft = upd(ib, dft)

In [ ]:
dft.to_pickle(fspath+'targets.pickle')

In [ ]:
pmask = (dft.right == 'P') & (dft.undPrice-dft.strike-dft.expPrice > dft.Fall)

df_prisky = pd.merge(dft[pmask][['symbol', 'strike', 'right', 'dte', 'undPrice', 'margin', 'optPrice', 'expPrice', 'Fall', 'Rise', 'expRom']], 
         pd.DataFrame((dft[pmask].undPrice-dft[pmask].strike-dft[pmask].expPrice)), on=dft[pmask].index).drop('key_0', 1)

cmask = (dft.right == 'C') & (dft.strike-dft.undPrice-dft.expPrice > dft.Rise)
df_crisky = pd.merge(dft[cmask][['symbol', 'strike', 'right', 'dte', 'undPrice', 'margin', 'optPrice', 'expPrice', 'Fall', 'Rise', 'expRom']], 
         pd.DataFrame((dft[cmask].strike-dft[cmask].undPrice-dft[cmask].expPrice)), on=dft[cmask].index).drop('key_0', 1)

df_risky = pd.concat([df_prisky, df_crisky]).reset_index(drop=True)

df_risky = df_risky.rename(columns={0: 'FallRise'})

In [ ]:
dft.head()

In [ ]:
minRom

In [4]:
%%time
dynamic(ib)

Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
commissionReport: No execution found for CommissionReport(execId='00016710.5cdccbae.01.01', commission=28.0955, currency='INR')
Started to throttle requests
Stopped to throttle requests
Started to throttle requests


NOTE: Dynamic calculation is to be run only an hour before markets open.

Entire target SELL trade will be placed


Stopped to throttle requests
Started to throttle requests
Error 0, reqId -1: Warning: Approaching max rate of 50 messages per second (44)
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle requests
Stopped to throttle requests
Started to throttle r

Wall time: 36 s


commissionReport: No execution found for CommissionReport(execId='00016710.5cdccbb9.01.01', commission=30.17675, currency='INR')


In [ ]:
ib.cancel